# Bsky integration

In [ ]:
//| export

import { BskyXRPC } from "@mary/bluesky-client";
import type {
  AppBskyEmbedExternal,
  AppBskyEmbedImages,
  AppBskyFeedDefs,
  AppBskyFeedPost,
  AppBskyRichtextFacet,
} from "@mary/bluesky-client/lexicons";

type UnwrapArray<T> = T extends (infer V)[] ? V : never;

export type Facet = AppBskyRichtextFacet.Main;
export type FacetFeature = UnwrapArray<Facet["features"]>;

type Thread = AppBskyFeedDefs.ThreadViewPost;
type Post = AppBskyFeedDefs.PostView;
type PostRecord = AppBskyFeedPost.Record;
type EmbedImages = AppBskyEmbedImages.View;
type EmbedExternal = AppBskyEmbedExternal.View;
type LinkFeature = AppBskyRichtextFacet.Link;
type MentionFeature = AppBskyRichtextFacet.Mention;
// type TagFeature = AppBskyRichtextFacet.Tag;

const rpc = new BskyXRPC({ service: "https://public.api.bsky.app" });

# Post URL to @at protocol URI

Figure out how to convert bsky.app URL to a URI we can use to fetch data. We
need to go from this:

```
https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d
```

To smth like this

```
at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
```

`did:plc:ubdeopbbkbgedccgbum7dhsh` is DID for callmephilip.com handle, which
needs to be resolved first

In [ ]:
//| export

const postURLToAtpURI = async (
  postUrl: string,
): Promise<[string, string]> => {
  const urlParts = new URL(postUrl);
  const pathParts = urlParts.pathname.split("/");
  const h = await rpc.get("com.atproto.identity.resolveHandle", {
    params: {
      handle: pathParts[2],
    },
  });

  return [`at://${h.data.did}/app.bsky.feed.post/${pathParts[4]}`, h.data.did];
};

In [ ]:
await postURLToAtpURI(
  "https://bsky.app/profile/jason.energy/post/3ldllxneijk2d",
);

[
  "at://did:plc:ga3wlji66r5mxqch6wh7nq4v/app.bsky.feed.post/3ldllxneijk2d",
  "did:plc:ga3wlji66r5mxqch6wh7nq4v"
]

# Grabbing thread data

Thread has a bunch of nested posts inside replies. Unwrap this into a list of
posts

In [ ]:
//| export

const unwrapThreadPosts = (
  thread: Thread,
): Post[] => {
  const posts: Post[] = [];

  // Add root post
  if (thread.post) {
    posts.push(thread.post);
  }

  // Recursively handle replies
  if (thread.replies) {
    thread.replies.forEach((reply) => {
      posts.push(...unwrapThreadPosts(reply as Thread));
    });
  }

  // Handle nested reply if present
  if ("parent" in thread && thread.parent) {
    posts.push(
      ...unwrapThreadPosts(thread.parent as Thread),
    );
  }

  return posts;
};

type ThreadData = {
  handle: string;
  posts: Post[];
};

export const downloadThread = async (
  postUrl: string,
): Promise<ThreadData> => {
  const [uri, handle] = await postURLToAtpURI(postUrl);
  const d = await rpc.get("app.bsky.feed.getPostThread", {
    params: {
      uri,
    },
  });
  return {
    handle,
    posts: unwrapThreadPosts(d.data.thread as Thread),
  };
};

In [ ]:
const { posts, handle } = await downloadThread(
  "https://bsky.app/profile/xenova.bsky.social/post/3ldlswuvnxs2i",
);

# Converting posts to MD

The most basic setup is when there is just a piece of text display. More complex
cases will have links, attachments

In [ ]:
//| export

// from https://github.com/mary-ext/skeetdeck/blob/aa0cb74c0ace489b79d2671c4b9e740ec21623c7/app/api/richtext/unicode.ts

const encoder = new TextEncoder();
const decoder = new TextDecoder();

interface UtfString {
  u16: string;
  u8: Uint8Array;
}

const createUtfString = (utf16: string): UtfString => {
  return {
    u16: utf16,
    u8: encoder.encode(utf16),
  };
};

const getUtf8Length = (utf: UtfString) => {
  return utf.u8.byteLength;
};

const sliceUtf8 = (utf: UtfString, start?: number, end?: number) => {
  return decoder.decode(utf.u8.slice(start, end));
};

// replace

In [ ]:
//| export

const externalEmbedToMarkdown = (
  embed: EmbedExternal,
  authorDid: string,
): string => {
  const { title, description, uri, thumb } = embed.external;

  console.log(embed);

  let thumbUrl: string | undefined;

  if (typeof thumb === "string") {
    thumbUrl = thumb;
  } else if (thumb) {
    thumbUrl =
      // @ts-ignore not sure why this is failing
      `https://cdn.bsky.app/img/feed_thumbnail/plain/${authorDid}/${thumb.ref.$link}@jpeg`;
  }

  const thumbImage = thumbUrl ? `![${title}](${thumbUrl})` : null;

  return [`### [${title}](${uri})`, description, thumbImage || ""]
    .map((l: string) => `> ${l}`)
    .join("\n");
};

In [ ]:
//| export

// from https://github.com/mary-ext/skeetdeck/blob/aa0cb74c0ace489b79d2671c4b9e740ec21623c7/app/api/richtext/segmentize.ts

export interface RichtextSegment {
  text: string;
  feature: FacetFeature | undefined;
}

const createSegment = (
  text: string,
  feature: FacetFeature | undefined,
): RichtextSegment => {
  return { text: text, feature: feature };
};

export const segmentRichText = (
  rtText: string,
  facets: Facet[] | undefined,
): RichtextSegment[] => {
  if (facets === undefined || facets.length === 0) {
    return [createSegment(rtText, undefined)];
  }

  const text = createUtfString(rtText);

  const segments: RichtextSegment[] = [];
  const length = getUtf8Length(text);

  const facetsLength = facets.length;

  let textCursor = 0;
  let facetCursor = 0;

  do {
    const facet = facets[facetCursor];
    const { byteStart, byteEnd } = facet.index;

    if (textCursor < byteStart) {
      segments.push(
        createSegment(sliceUtf8(text, textCursor, byteStart), undefined),
      );
    } else if (textCursor > byteStart) {
      facetCursor++;
      continue;
    }

    if (byteStart < byteEnd) {
      const subtext = sliceUtf8(text, byteStart, byteEnd);
      const features = facet.features;

      if (features.length === 0 || subtext.trim().length === 0) {
        segments.push(createSegment(subtext, undefined));
      } else {
        segments.push(createSegment(subtext, features[0]));
      }
    }

    textCursor = byteEnd;
    facetCursor++;
  } while (facetCursor < facetsLength);

  if (textCursor < length) {
    segments.push(
      createSegment(sliceUtf8(text, textCursor, length), undefined),
    );
  }

  return segments;
};

const processFacets = (text: string, facets: Facet[] | undefined): string => {
  return segmentRichText(text, facets).reduce((acc, segment) => {
    const { text, feature } = segment;

    if (feature?.$type === "app.bsky.richtext.facet#link") {
      return acc + `[${text}](${feature.uri})`;
    } else if (feature?.$type === "app.bsky.richtext.facet#mention") {
      return acc + `[${text}](https://bsky.app/profile/${feature.did})`;
    }

    return acc + segment.text;
  }, "");
};

export const postToMd = (post: Post, handle: string): string => {
  const record = post.record as PostRecord;
  const text = record.text;
  let richtext = text;
  let embeds = "";

  richtext = processFacets(text, record.facets);

  if (post.embed) {
    if (post.embed.$type === "app.bsky.embed.images#view") {
      const e = post.embed as EmbedImages;
      for (const image of e.images) {
        embeds += `![${
          image.alt || "no image description"
        }](${image.fullsize})\n`;
      }
    } else if (post.embed.$type === "app.bsky.embed.external#view") {
      embeds += externalEmbedToMarkdown(
        post.embed as EmbedExternal,
        handle,
      );
    }
  }

  const [d, t] = record.createdAt.split("T");
  const [h, m] = t.split(":");

  return [
    `> [${post.author.displayName} - @${post.author.handle}](https://bsky.app/profile/${post.author.handle}) **${d} ${
      [h, m].join(
        ":",
      )
    }**`,
    ...richtext.split("\n").filter((l: string) => l.trim() !== ""),
    embeds,
  ].join("\n\n");
};

// await Deno.jupyter.display(
//   {
//     "text/markdown": postToMd(posts[4], handle),
//   },
//   { raw: true }
// );

Support links inside posts

In [ ]:
//| export

export const downloadPostToMd = async (postUrl: string): Promise<string> => {
  const { posts, handle } = await downloadThread(postUrl);

  return posts.map((post) => postToMd(post, handle)).join("\n\n");
};

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": await downloadPostToMd(
      "https://bsky.app/profile/callmephilip.com/post/3ldl55b3h5k2r",
    ),
  },
  { raw: true },
);

{
  "$type": "app.bsky.embed.external#view",
  external: {
    uri: "https://github.com/psky-atp",
    title: "Picosky",
    description: "Picosky has 2 repositories available. Follow their code on GitHub.",
    thumb: "https://cdn.bsky.app/img/feed_thumbnail/plain/did:plc:mdjhvva6vlrswsj26cftjttd/bafkreibx3trzsxshp3snn64ie2wxh3mpv4qqilf6m3lzfx7lk6mnrbjb4a@jpeg"
  }
}


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-18 10:21**

1/* I am playing with an idea of repurposing tinychat ([github.com/callmephilip...](https://github.com/callmephilip/tinychat)) into an open source version of discord running on top to atprotocol. So much of valuable community contributions and discussions are currently trapped in the walled gardens of discord.



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-18 10:21**

2/* This data should be indexable, searchable and actionable. Public goods services should be built around it to serve communities. It feels like Discord is misused (in the sense of being used for things it was NOT designed for) due to its popularity



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-19 14:32**

3/* This system should support both free form chitchat that people enjoy over Discord and also a more structured "thread" based discussions similar to traditional forums. How do you structure UX to nudge people towards threads when appropriate. Discord has "Create thread" but few people use it



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-19 14:32**

4/* Should the system attempt to cluster comments/messages into threads and do some house keeping in real time?



> [Laurens - @laurenshof.online](https://bsky.app/profile/laurenshof.online) **2024-12-19 10:49**

interesting project, tinychat looks cool

if you do, might be worth looking at picosky for some inspiration, who also build a chatting service on top of atproto

[github.com/psky-atp](https://github.com/psky-atp)

> ### [Picosky](https://github.com/psky-atp)
> Picosky has 2 repositories available. Follow their code on GitHub.
> ![Picosky](https://cdn.bsky.app/img/feed_thumbnail/plain/did:plc:mdjhvva6vlrswsj26cftjttd/bafkreibx3trzsxshp3snn64ie2wxh3mpv4qqilf6m3lzfx7lk6mnrbjb4a@jpeg)

> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-19 10:52**

absolutely! thanks for the tip. going to keep an eye on it

